In [1]:
import tensorflow as tf
from scipy.signal import filtfilt as scipy_filtfilt
import numpy as np

# Filt-Filt operation definition

- FeedBack filter components `[M]`:   $A = a_0, a_1, a_2 \dots a_M$, where $a_0 = 0$ by definition
- FeedForward filter components`[N]`:   $B = b_0, b_1, b_2 \dots b_N$

```
y(t) = x(t)*b(0) + x(t-1)*b(1) + x(t-2)*b(2) ... x(t-M)*b(N)
                 - y(t-1)*a(1) - y(t-2)*a(2) ... y(t-N)*a(M)
```

$$
y_t = \sum_{i=0}^{N} x_{t-i} \cdot b_{i} - \sum_{i=1}^{M} y_{t-i} \cdot a_{i}
$$


```
y(t) = x[t-N : t] * b[::-1] - y[t-M : t-1] * a[::-1]
```

for faster computation let $s$ be $s := x \ast b$, where $\ast$ is the convolution operator, since all values of $x$ is known at computation time

```
y(t) = h[t] - y[y-t : t-1] * a[::-1]
```


# Final solution

with tf loop... instead of queues

In [161]:
# In Part 2 the queue released 3 elements every time, but enqueued only 1 in return
# Now for rapid rebuilding I put everythin into one cell
import tensorflow as tf
import numpy as np

tf.reset_default_graph()
sess = tf.InteractiveSession()

filter_size_upper_bound = 100
CAPACITY = 300 * filter_size_upper_bound

# [batch_size, sequnce_length]
x = tf.placeholder(tf.float32, shape=[None, None], name='x_vector')

# [filter_length(N/M), num_filters]
a = tf.placeholder(tf.float32, shape=[15, 10], name='a_vector')
b = tf.placeholder(tf.float32, shape=[20, 10], name='b_vector')

batch_size = tf.shape(x)[0]
sequence_length = tf.shape(x)[1]
N = tf.shape(b)[0]
M = tf.shape(a)[0]
num_filters_a = tf.shape(a)[1]
num_filters_b = tf.shape(b)[1]
# filter_assert = tf.assert_equal(num_filters_a, num_filters_b)
num_filters = num_filters_a
'''y = tf.Variable([], trainable=False, validate_shape=False, name='y_vector')
y_init = tf.zeros(dtype=tf.float32, 
                  shape=[sequence_length, batch_size, num_filters],
                  name='y_zeros_init')
y_init = tf.assign(y, y_init)
'''

y = tf.zeros(dtype=tf.float32, 
             shape=[batch_size, M, num_filters],
             name='y_zeros_init')

x_padded = tf.pad(x, [[0, 0], [N-1, 0]])

s = tf.nn.conv1d(
    value=x_padded[..., None], 
    filters=b[::-1, None, :], 
    stride=1, 
    padding='VALID',
    name='s_vector')

def prod(a, b, **kwargs):
    return tf.reduce_sum(a * b, axis=1, **kwargs)

In [173]:
i = list(range(10))

i[-1:3:-1]

[9, 8, 7, 6, 5, 4]

In [196]:
def body(y, s, a, t):
    y_curr = s[:, t] - prod(y[:, t:(t+M)], [a], name='curr_y')
    y = tf.concat([y, y_curr[:, None, :]], axis=1, name='y')
    
    t += 1
    return y, s, a, t

def cond(y, s, a, t):
    return  t < sequence_length


wl = tf.while_loop(cond, body, [y, s, a, 0], parallel_iterations=1, back_prop=False)
filt = tf.pad(wl[0][:, -1:M-1:-1], [[0, 0], [M, 0], [0, 0]])
wl = tf.while_loop(cond, body, [filt, s, a, 0], parallel_iterations=1, back_prop=False)
filtfilt = wl[0][:, -1:M-1:-1]



In [206]:
#tf.global_variables_initializer().run()
test_feed = {
    x: np.random.randn(100, 200),
    a: np.random.randn(15, 10)*.001,
    b: np.random.randn(20, 10)*.001
}
res = sess.run(filtfilt, test_feed)
res.shape

(100, 400, 10)

# Note: 

## The following are my attempts, which started with a lot of misunderstanding, but with progress everything cleared out

# First attempt

In [2]:
def resetSession():
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    return sess

In [3]:
sess = resetSession()
# Input
x = tf.placeholder(tf.float32, shape=[None], name='x_vector')

# Feedback filter
a = tf.placeholder(tf.float32, shape=[None], name='a_vector')
# Feedforward filter
b = tf.placeholder(tf.float32, shape=[None], name='b_vector')

N = tf.shape(b)[0]
M = tf.shape(a)[0]

In [4]:

# Support vector
# Default convolution works like this
# [... 1  2  3  4  5...]
#      *  *  *  *  *
# [    a  b  c  d  e    ]
#            +
#         result

# But this time we need the rightmost value as a result
# [... 1  2  3  4  5...]
#      *  *  *  *  *
# [    a  b  c  d  e    ]
#                  +
#               result
#
# So we use padding like a Jedi, and use VALID convolution

x_padded = tf.pad(x, [[N-1, 0]])

# value[None, :, None] stands for [Batch, Sample, Channels]
# In the basic example Batch = Channel = 1
# filters[::-1, None, None] stands for [Filter_size(inverted), In_size, Out_size]
# In the basic example In_size = Out_size = 1
s = tf.nn.conv1d(value=x_padded[None, :, None], filters=b[::-1, None, None], stride=1, padding='VALID')
s = tf.squeeze(s, name='s_vector')


In [6]:
# Watch out, the filter is now inverted
x_test = list(range(1,7))
b_test = [0, 0, 0, 1]
s.eval({x:x_test, b:b_test})

# [... 1  2  3  4  5...]
#      *  *  *  *  *
# [    e  d  c  b  a   ]
#                  +
#               result

array([ 0.,  0.,  0.,  1.,  2.,  3.], dtype=float32)

In [ ]:
b_test = [1, 0, 0]
s.eval({x:x_test, b:b_test})

# PART 2

## The Support vector $s$

Last cell in this part has some problems, will be rewrited in Part 3

In [ ]:
# Output
# Y values can be evaluated one-by-one, so as soon as we evaluate the t^th value
# We push it to the back of the queue, and we always take as many values
# as long is the Feedback filter vector is, for efficiently calculating inner product

# Capacity is the upper bound of the size of the filter, 
# Since the queue needs to be able to dequeue filter sized vectors at once.

filter_size_upper_bound = 100
CAPACITY = 3 * filter_size_upper_bound
# Since we could enqueue multiple tensors we use 
# listed `shapes` and `dtypes` but that is only for syntax autofellatio
# Basically we will enqueue and dequeue scalar values (shape=[], dtype=tf.float32)
y_queue = tf.FIFOQueue(CAPACITY, dtypes=[tf.float32], shapes=[[]], name='y_fifo')

# TF QUEUE mechanism demonstration

**Supplementary reading [TF guide on queues](https://www.tensorflow.org/programmers_guide/threading_and_queues)**

![demo](https://www.tensorflow.org/images/IncremeterFifoQueue.gif)

In [ ]:
# We initialize the queue with zeros to mimic padding
init = y_queue.enqueue_many(tf.zeros([N]))
# y_deq is technically y[t-M : t-1] 
# which will be used for the inner product operation
y_deq = y_queue.dequeue_many(N)

In [ ]:
print(sess.run([init, y_deq], {b:b_test}))
# The result is
# [None, array([ 0.,  0.,  0.], dtype=float32)]
# because evaluating the init operator does not return any value
# and evaluating the y_prev just returns our initialized values
# be careful! now the queue is empty again

# TRY ME!
# print(sess.run([y_queue.enqueue((1)), init, y_prev], {b:b_test}))
# now the queue is not empty, one zero is still left in it
# be careful! it remembers its state within a Session
# print(sess.run([y_queue.enqueue((1)), init, y_prev], {b:b_test}))

In [ ]:
# a single vector product... a bit too verbose
print(sess.run(tf.reduce_sum(a * b), {a:[1, 2, 3], b:[1, 10, 100]}))
# reduce it
def prod(a, b, **kwargs):
    return tf.reduce_sum(a * b, **kwargs)

In [ ]:
# evaluating y_1, using s_t with zero indexing is s[t-1]
# Now we write by hand the first step, 
# but this is not necessary
y_1 = s[0] - prod(y_deq, a) 

In [ ]:
x_test = list(range(1, 5))
b_test = [1, 0, 0]
a_test = [1, 0, 0]
test_feed = {x:x_test, b:b_test, a:a_test}


sess.run([init, s, y_deq, y_1], test_feed)

In [ ]:
# now evaluate a few more y_t
# and see that to do it by hand is very annoying
y_1 = s[0] - prod(y_deq, a) 
enq1 = y_queue.enqueue(y_1)

y_2 = s[1] - prod(y_deq, a)
enq2 = y_queue.enqueue(y_2)

y_3 = s[2] - prod(y_deq, a)
enq3 = y_queue.enqueue(y_3)

y_4 = s[3] - prod(y_deq, a)

explicit_fetch_list = [
    init,
    enq1,
    enq2,
    enq3,
    y_4,
]
sess.run(explicit_fetch_list, test_feed)

In [ ]:
# IMPORTANT NOTE!!!
# The order of graph definition here is not relevant
# These operations are not executed by calling the lines above
# the evaluation order is determined in the _fetch_list
# 
# fetch is the operation when you evaluate a tensor
# fetch its value (in the form of a numpy array) 
# from the computational graph
#
# The python variables here can be represented as
# C++ pointers... so here each operator is pointed by y_i
#
# If I wrote simply y instead
# like this

y = s[0] - prod(y_deq, a) 
enq1 = y_queue.enqueue(y)

y = s[1] - prod(y_deq, a)
enq2 = y_queue.enqueue(y)

y = s[2] - prod(y_deq, a)
enq3 = y_queue.enqueue(y)

y = s[3] - prod(y_deq, a)

impl_fetch_list = [
    y_deq,
    init,
    enq1,
    enq2,
    enq3,
    y
]

# In this case y always points to the latest operation/tensor we defined
# It is still working because enq1, enq2... are linked together with
# different y_t tensors

# In order to make the computation valid
# We need to enqueue in the right order

sess.run(impl_fetch_list, test_feed)


# Part 3

**Correcting Part 2**

Automatizing the operations, tensorflow CONTROLFLOW operations

In [1]:
# In Part 2 the queue released 3 elements every time, but enqueued only 1 in return
# Now for rapid rebuilding I put everythin into one cell
import tensorflow as tf
import numpy as np

tf.reset_default_graph()
sess = tf.InteractiveSession()

filter_size_upper_bound = 100
CAPACITY = 300 * filter_size_upper_bound


x = tf.placeholder(tf.float32, shape=[None], name='x_vector')
y = tf.placeholder(tf.float32, shape=[None], name='y_vector')
#a = tf.placeholder(tf.float32, shape=[None], name='a_vector')
#b = tf.placeholder(tf.float32, shape=[None], name='b_vector')

a = tf.Variable([1., 0., 0.], trainable=False)
b = tf.Variable([1., 0., 0.], trainable=False)


N = tf.shape(b)[0]
M = tf.shape(a)[0]

x_padded = tf.pad(x, [[N-1, 0]])

s = tf.nn.conv1d(value=x_padded[None, :, None], filters=b[::-1, None, None], stride=1, padding='VALID')
s = tf.squeeze(s, name='s_vector')

y_q = tf.FIFOQueue(CAPACITY, dtypes=[tf.float32], shapes=[[]], name='y_fifo')
#init_zeros = tf.Print(tf.zeros([N]), [y_q.size()], message='ENQ - init - Nb of elements left in queue:')
y_init = y_q.enqueue_many(tf.zeros([N]))
y_deq = y_q.dequeue_many(N)
#y_deq = tf.Print(y_deq, [y_q.size()], message='DEQ - Nb of elements left in queue:')


test_feed = {
    x:list(range(1,7)),
    a:[1, 0, 0],
    b:[1, 0, 0]
}

def prod(a, b, **kwargs):
    return tf.reduce_sum(a * b, **kwargs)

# IMPORTANT

Cells below are behaving differently

# Clearing up

In [2]:
# it seems that "float" can be used as well...
s_q = tf.FIFOQueue(25000, 'float', shapes=[[]], name='support_fifo')
# We don't want to slice each time we evaluate y
# So we push the whole support vector into the queue
# If running in parallel then he capacity can be exceeded while enqueueing
# Capacity is about the maximum number of elements actually in the queue
# hanging enqueues will activate when the q is dequeued

s_init = s_q.enqueue_many(s)
s_deq = s_q.dequeue()
# s_deq = tf.Print(s_deq, [s_deq, s_q.size()], message='DEQ - s_deq | s_q.size')

In [12]:
# actual evaluation
y_curr = s_deq - prod(y_deq, a)
#y_curr = tf.Print(y_curr, [y_curr, y_q.size()], message='DEQ - y_curr | y_q.size')
# we need to feed back the last M-1 values
# shifting the window, dropping the oldest values
# which is the rightmost, lowest index
y_prev = y_deq[1:]
y_feedback = tf.concat([y_prev, [y_curr]], axis=0, name='y_feedback')
#y_feedback = tf.Print(y_feedback, [y_feedback], message='ENQ - y_feedback')
y_feedback = y_q.enqueue_many(y_feedback)

In [13]:
'''
Create an op that groups multiple operations_

When this op finishes, all ops in input have finished. This op has no output.
'''
init = tf.group(y_init, s_init, name='init')

In [14]:
tf.global_variables_initializer().run()
init.run({x:np.random.randn(10000)})


In [8]:
i = 0
while s_q.size().eval() > 0:
    if i% 100 == 0: print('%d\r'%i, end='', flush=True)
    i += 1
    y_feedback.run()

KeyboardInterrupt: 

In [15]:
#y = tf.Variable([0.]*3, trainable=False, name='y')

# Final solution

with tf loop... instead of queues

In [1]:
# In Part 2 the queue released 3 elements every time, but enqueued only 1 in return
# Now for rapid rebuilding I put everythin into one cell
import tensorflow as tf
import numpy as np

tf.reset_default_graph()
sess = tf.InteractiveSession()

filter_size_upper_bound = 100
CAPACITY = 300 * filter_size_upper_bound


x = tf.placeholder(tf.float32, shape=[None], name='x_vector')
y = tf.placeholder(tf.float32, shape=[None], name='y_vector')
t = tf.placeholder_with_default(0, shape=[], name='time')
a = tf.placeholder(tf.float32, shape=[None], name='a_vector')
b = tf.placeholder(tf.float32, shape=[None], name='b_vector')

#a = tf.Variable([1., 0., 0.], trainable=False)
#b = tf.Variable([1., 0., 0.], trainable=False)


N = tf.shape(b)[0]
M = tf.shape(a)[0]

x_padded = tf.pad(x, [[N-1, 0]])

s = tf.nn.conv1d(value=x_padded[None, :, None], filters=b[::-1, None, None], stride=1, padding='VALID')
s = tf.squeeze(s, name='s_vector')



def prod(a, b, **kwargs):
    return tf.reduce_sum(a * b, **kwargs)

In [2]:
def body(s, y, a, t):
    y_curr = s[t] - prod(y, a)
    y_prev = y[1:]
    y = tf.concat([y_prev, [y_curr]], axis=0, name='y_feedback')
    #y = tf.Print(y, [y, t], message='ENQ - y_feedback || t')
    t += 1
    return s, y, a, t

def cond(s, y, a, t):
    return  t < tf.shape(x)[0]


wl = tf.while_loop(cond, body, [s, y, a, t], parallel_iterations=1, back_prop=False)


In [7]:
#tf.global_variables_initializer().run()
test_feed = {
    x: np.random.randn(20000),
    y: [0]*10,
    a: np.eye(10)[0],
    b: np.eye(10)[0]
}
res = sess.run([wl], test_feed)
